In [9]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

from typing import TypedDict, Annotated, Literal
from pydantic import BaseModel, Field

from langchain.output_parsers import StructuredOutputParser, ResponseSchema, PydanticOutputParser
from langchain_core.output_parsers import StrOutputParser

from langchain.schema.runnable import RunnableParallel, RunnablePassthrough, RunnableBranch, RunnableLambda



In [4]:
load_dotenv()

True

In [5]:
llm=ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite",temperature=0.6)
response=llm.invoke("how are you?")
print(response.content)

I am a large language model, trained by Google. I am functioning as expected.


In [95]:
#prompt templates

template=PromptTemplate(template="Tell me interesting facts about this country: {country}",input_variables=["country"])
country=input("enter counrty name: ")
prompt=template.invoke({"country":country})
result=llm.invoke(prompt)
print(result.content)

India is a land of incredible diversity and rich history, offering endless fascinating facts. Here are some that might pique your interest:

**Ancient Wonders & Inventions:**

*   **The Invention of Chess:** The ancient Indian game of Chaturanga is considered the precursor to modern chess, dating back to the 6th century AD.
*   **The Number Zero:** The concept of zero as a placeholder and its mathematical significance were first developed in India. This revolutionized mathematics and science.
*   **Ayurveda:** This ancient Indian system of medicine, which focuses on holistic well-being, is over 5,000 years old and is still practiced today.
*   **The First Planned Cities:** The Indus Valley Civilization, which flourished in ancient India, had some of the world's earliest urban planning, with sophisticated drainage systems and grid-like city layouts.
*   **The Kumbh Mela:** This Hindu pilgrimage is the largest gathering of people on Earth, with millions of devotees converging every few y

In [51]:
# Chat Prompt Template

chat_template=ChatPromptTemplate.from_messages(['System',"You are a helpful stock market assistant",'Human',f"Tell me about stock price and market cap of {company}"])
company=input("Enter comapny name: ")
prompt=chat_template.invoke({"company":company})
result=llm.invoke(prompt)
print(result.content)

I can help you with that! However, I need to clarify that I **cannot provide real-time stock prices or market capitalization data**. This information changes constantly throughout the trading day and requires access to live market feeds.

To get the most up-to-date information on TCS's stock price and market cap, I recommend checking reliable financial news websites or stock tracking platforms. Here are some excellent resources:

**Where to Find Real-Time TCS Stock Information:**

*   **Financial News Websites:**
    *   **Google Finance:** Simply search for "TCS stock" or "Tata Consultancy Services stock price."
    *   **Yahoo Finance:** Similar to Google Finance, search for "TCS.NS" (for the Indian stock exchange) or "Tata Consultancy Services."
    *   **Bloomberg:** A comprehensive source for financial data.
    *   **Reuters:** Another reputable source for financial news and data.
    *   **Moneycontrol:** A popular Indian financial portal.
    *   **The Economic Times:** A leadi

In [70]:
# Chat Prompt Template with the history of the conversation
chat_prompt=ChatPromptTemplate.from_messages([('system',"You are a Helpful coking assistant"),('human',f"Tell me the recipe of {dish} in 7 points")])
chat_history=[]
dish=input("Enter the dish name: ")
prompt=chat_prompt.invoke({"dish":dish})
chat_history.append(HumanMessage(content=f"Tell me the recipe of {dish} in 7 points"))
result=llm.invoke(prompt)
chat_history.append(AIMessage(content=result.content))
print(result.content)

# history of the converation
# For better result of the conversation we have to iterate the conversation
print(chat_history)



Here's a simple pasta recipe in 7 points:

1.  **Boil Water & Salt:** Fill a large pot with plenty of water, add a generous pinch of salt, and bring it to a rolling boil over high heat.
2.  **Add Pasta:** Carefully add your desired amount of dried pasta to the boiling water, stirring occasionally to prevent sticking.
3.  **Cook Al Dente:** Cook the pasta according to package directions until it's tender but still has a slight bite (al dente).
4.  **Reserve Pasta Water:** Before draining, scoop out about a cup of the starchy pasta water and set it aside.
5.  **Drain Pasta:** Drain the cooked pasta thoroughly in a colander.
6.  **Combine with Sauce:** Return the drained pasta to the pot (or a large bowl) and immediately toss it with your favorite sauce, adding a splash of the reserved pasta water to help the sauce coat the pasta.
7.  **Serve:** Plate the pasta and serve hot, garnished with cheese or fresh herbs if desired.
[HumanMessage(content='Tell me the recipe of pasta in 7 points', 

In [ ]:
# Structured Output by the help of TypedDict and Literal
class Review(TypedDict):
     summary:str
     sentiment:Literal["Neutral", "Horror","Scientific","Action","Romantic","Comedy","Drama"]
review_prompt=ChatPromptTemplate.from_messages([('system',"You are a helpful movie review assistant"),('human',f"give me summary and sentiment of the movie: {movie}")])
movie=input("Enter movie name: ")
prompt=review_prompt.invoke({"movie":movie})
structured_output_llm=llm.with_structured_output(Review)

result=structured_output_llm.invoke(prompt)
print(result)

{'summary': 'This movie is about a group of people who get trapped in a giant sinkhole and have to find a way to escape.', 'sentiment': 'Horror'}


In [92]:
# Structured Output parser
Schemas=[
    ResponseSchema(name="summary",description="summary of the movie in 50 words"),
    ResponseSchema(name="name",description="name of the director of the movie")
]
output_parser=StructuredOutputParser.from_response_schemas(Schemas)
format_instructions=output_parser.get_format_instructions()
template=PromptTemplate(template="give me summary and name of the director of the movie: {movie} in Output format \n {format_instructions}",input_variables=["movie"],partial_variables={"format_instructions":format_instructions})
prompt=template.invoke({"movie":"Fir Hera Pheri"})
result=llm.invoke(prompt)
final=output_parser.parse(result.content)
print(final)

{'summary': 'Three men, struggling with debt, accidentally receive a ransom call meant for a millionaire. They hatch a plan to impersonate the kidnappers and extort money, leading to a series of hilarious mishaps and escalating chaos as their scheme unravels.', 'name': 'Neeraj Vora'}


In [ ]:
# Chain implementation in the structured output parser
chain=template|llm|output_parser
result=chain.invoke({"movie":"Jai Bhim"})
print(result)

{'summary': 'Jai Bhim is a powerful legal drama based on a true story. It follows lawyer Chandru as he fights for justice for a tribal woman falsely accused of theft by the police. The film exposes systemic injustice and caste discrimination in rural Tamil Nadu, highlighting the struggle for human rights.', 'name': 'T. J. Gnanavel'}


In [98]:
# Pydantic Output Parser
class chemist(BaseModel):
    usage: str =Field(description="What is the use of given chemical compound or the medician in one line.")
    formula: str =Field(description="What is the chemical formula of given medician or chemical compound.")
output_parser=PydanticOutputParser(pydantic_object=chemist)
template=PromptTemplate(template="Give me the usage and formula of the chemical compound or medician: {medician} in the format \n {format_instructions}",input_variables=["medician"],partial_variables={"format_instructions":output_parser.get_format_instructions()})

chain=template|llm|output_parser
result=chain.invoke({"medician":"Paracetamol"})
print(result)

usage='Paracetamol is used to treat mild to moderate pain and reduce fever.' formula='C8H9NO2'


In [106]:
# CHAINS
# Parallel Chain
prompt1=PromptTemplate(template="Tell me the GDP on the basis of PPP of the country: {country} in the year 2015",input_varibales=["country"])
prompt2=PromptTemplate(template="Tell me the GDP on the basis of PPP of the country: {country} in the year 2020",input_varibales=["country"])
prompt3=PromptTemplate(template="Tell me weather the country's economy was improved or not in between 2015 to 2020 on the basis of {gdp_2015} and {gdp_2020}",input_varibales=["gdp_2015","gdp_2020"])
parser=StrOutputParser()

chain=RunnableParallel({
    'gdp_2015':prompt1|llm|parser,
    'gdp_2020':prompt2|llm|parser
})

merged=prompt3|llm|parser
chain=chain|merged
result=chain.invoke({"country":"India"})
print(result)


Yes, based on the provided figures for India's GDP based on Purchasing Power Parity (PPP), the country's economy **improved** between 2015 and 2020.

Here's why:

*   **2015 GDP (PPP):** Approximately $8.7 trillion USD
*   **2020 GDP (PPP):** Approximately $10.26 trillion USD

The GDP (PPP) increased from $8.7 trillion to $10.26 trillion USD. This signifies an **increase in the overall economic output** of the country when adjusted for differences in the cost of living. A higher GDP (PPP) generally indicates a stronger and more productive economy.


In [22]:
# CHAINS
# Conditional Chain

parser=StrOutputParser()

class Review(BaseModel):
    sentiment:Literal["Positive","Negative","Neutral"]=Field(description="Sentiment of the provided review")
        
pydantic_parser=PydanticOutputParser(pydantic_object=Review)
prompt1=PromptTemplate(template="Tell me the sentiment of the review: {review} in the format \n {format_instructions}",input_variables=["review"],partial_variables=({"format_instructions":pydantic_parser.get_format_instructions()}))

prompt2=PromptTemplate(template="Give me a response for the positive review: {review}",input_variables=["review"])
prompt3=PromptTemplate(template="Give me a response for the negative review: {review}",input_variables=["review"])

classifier=prompt1|llm|pydantic_parser




# RunnableBranch does not accept dicts ({"sentiment": "Positive"}) as conditions.
# It expects callables (lambda functions) that return True or False. So the following code is incorrect:
# branch_chain=RunnableBranch(
#     ({"sentiment":"Positive"},prompt2|llm|parser),                                                   
#     ({"sentiment":"Negative"},prompt3|llm|parser),
#     ({"sentiment":"Neutral"},RunnablePassthrough()),
#     # ({"sentiment":""},"could not detect sentiment")
# )





branch_chain=RunnableBranch(
    (lambda x: x.sentiment=="Positive",prompt2|llm|parser),
    (lambda x: x.sentiment=="Negative",prompt3|llm|parser),
    (lambda x: x.sentiment=="Positive",RunnablePassthrough()),
    RunnableLambda(lambda x: "could not detect sentiment")
)
review="The product was really good and I love the display of the product. Also the camera quality is very good."
chain=classifier|branch_chain
result=chain.invoke({"review":review})
print(result)


Here are several options for responding to a positive review, categorized by tone and focus. Choose the one that best fits your brand's voice and the specific review:

**Short, Sweet & Enthusiastic:**

*   "Thank you so much for your wonderful review! We're thrilled you enjoyed [mention something specific if possible, e.g., your experience, our product]!"
*   "Wow, thank you for the amazing feedback! We're so happy to hear you had a great time."
*   "We're delighted to receive your positive review! Thanks for taking the time to share your thoughts."
*   "Your kind words mean a lot to us! Thank you for the fantastic review."

**Slightly More Detailed & Appreciative:**

*   "Thank you for your incredibly positive review! We're so pleased to know that you [mention something specific, e.g., loved our service, found our product helpful, had a memorable experience]. We appreciate your business!"
*   "We're absolutely thrilled to read your fantastic review! It's wonderful to hear that you had

In [24]:
chain.get_graph().print_ascii()

      +-------------+      
      | PromptInput |      
      +-------------+      
             *             
             *             
             *             
    +----------------+     
    | PromptTemplate |     
    +----------------+     
             *             
             *             
             *             
+------------------------+ 
| ChatGoogleGenerativeAI | 
+------------------------+ 
             *             
             *             
             *             
 +----------------------+  
 | PydanticOutputParser |  
 +----------------------+  
             *             
             *             
             *             
        +--------+         
        | Branch |         
        +--------+         
             *             
             *             
             *             
     +--------------+      
     | BranchOutput |      
     +--------------+      
